
# NVIDIA Meeting Recorder Agent: 01-Initialize and Setup Agent

In this blueprint, we'll build a meeting recorder agent capable of finding appointments on your Google Calendar, recording the meetings, and sending the recordings via email.

We’ll be using the xpander.ai Python SDK and NVIDIA NIM to bring this agent to life. The xpander SDK handles the overall workflow and orchestration, while NVIDIA NIM powers the agent's LLM.

To get started, we'll need to set up a few prerequisites:
- xpander API key

- Google Calendar integration

- NVIDIA NIM API key

## Setting Up Your xpander API Key
To get your xpander API key, go to https://app.xpander.ai/, and you'll land on this page:

![img_1](images/img_1.png)

In the sidebar, navigate to "Settings" > "API Keys". Copy your xpander API key and paste it into the `.env` file included in this repo.

## Google Calendar Integration
A key feature of our agent is the ability to check your Google Calendar for scheduled meetings. To make that happen, xpander needs permission to access your calendar.

To set it up, click on "Interfaces" in the sidebar. You’ll see a list of available connectors for popular SaaS platform offered by xpander.ai.

![img_2](images/img_2.png)

Click on Google Calendar, then select "Sign In with Google Calendar". Then, follow the prompts to grant xpander access to your calendar.

![img_3](images/img_3.png)

## Setting Up NVIDIA NIM API Key

You can get the NVIDIA NIM API key for free. Just head over to https://build.nvidia.com/models and select llama-3.1-405b-instruct.

![img_4](images/img_4.png)

Click on "Get API Key" in the top-right corner, then copy your NVIDIA NIM API key and paste it into the `.env` file in this repo.

## Initializing the AI Agent with xpander

Now that we’ve completed all the necessary setup steps, we can initialize our agent using xpander.

First, let’s load the xpander API key from the `.env` file:

In [1]:
"""AI Meeting Recorder Agent registration script for Xpander Cloud."""
import os
import json
from xpander_sdk import XpanderClient, GraphItem, AgentGraphItemType
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key is available
xpander_api_key = os.environ.get("XPANDER_API_KEY")
if not xpander_api_key:
    print("❗️ Missing XPANDER_API_KEY in .env file")
else:
    print("✅ XPANDER_API_KEY loaded successfully")

# Initialize Xpander client
xpander_client = XpanderClient(api_key=xpander_api_key)

✅ XPANDER_API_KEY loaded successfully


Now, we’ll either initialize a new agent (if one hasn’t been created yet) or retrieve the agent ID if it was previously initialized.

In [2]:
# Get existing agent ID from environment or create a new agent
agent_id = os.environ.get("XPANDER_AGENT_ID")
agent_name = "AI Meeting Recorder Agent"

try:
    if agent_id:
        agent = xpander_client.agents.get(agent_id=agent_id)
        print(f"Using existing agent: {agent.name} (ID: {agent.id})")
    else:
        agent = xpander_client.agents.create(name=agent_name)
        agent_id = agent.id
        
        # Update .env with new agent ID ## Optional
        with open(".env", "r") as f:
            env_content = f.read()
        with open(".env", "w") as f:
            if "XPANDER_AGENT_ID" in env_content:
                f.write("\n".join([line if not line.startswith("XPANDER_AGENT_ID=") 
                                 else f"XPANDER_AGENT_ID={agent.id}" for line in env_content.splitlines()]))
            else:
                f.write(f"{env_content}\nXPANDER_AGENT_ID={agent.id}")
        print(f"Created new agent: {agent.name} (ID: {agent.id})")
except Exception as e:
    print(f"❌ Error with agent: {e}")

b'{"stdout":"bG9hZGluZyBhZ2VudCA3NWFmYzg3Yy01ZjYyLTQwYWItOGM1ZS0yNjFmYjNmNGE2NTQK"}\n'
b'{"stdout":"bG9hZGluZyBhZ2VudCA3NWFmYzg3Yy01ZjYyLTQwYWItOGM1ZS0yNjFmYjNmNGE2NTQK"}\n'
b'{"stdout":"QWdlbnQgbG9hZGVkIGZyb20gY2FjaGUK"}\n'
b'{"stdout":"bG9hZGluZyBhZ2VudCA3NWFmYzg3Yy01ZjYyLTQwYWItOGM1ZS0yNjFmYjNmNGE2NTQK"}\n'


Created new agent: AI Meeting Recorder Agent (ID: 75afc87c-5f62-40ab-8c5e-261fb3f4a654)


And just like that, we’ve initialized our AI agent with xpander!

If you check the contents of your `.env` file, you’ll see that your xpander agent ID has been saved there. We’ll need this ID anytime we want to access or make changes to this specific agent.

You can also use the agent ID to view a visualization graph of your agent in xpander’s Workbench using the following link:

In [3]:
# Print summary
print(f"\nAgent initialization complete!")
print(f"Go to: https://app.xpander.ai/agents/{agent.id}")


Agent initialization complete!
Go to: https://app.xpander.ai/agents/75afc87c-5f62-40ab-8c5e-261fb3f4a654


If you click the link above, you should see a visualization similar to this:

![img_5](images/img_5.png)

## Adding Tools to AI Agent

In its current state, our agent is just a general chatbot with no special capabilities. Therefore, we’ll improve our agent's capabilities by adding operators (or tools) from a couple of connectors.

As you've seen in the above section, xpander offers a wide range of connectors for popular SaaS platforms like Google WorkSuitespace, Slack, Zillow, Notion, and more. These connectors can be easily integrated into your agent, giving it access to those platforms and allowing you to build a powerful AI agent.

Let’s explore the available connectors that are ready to be used by our agent.



In [4]:
agent = xpander_client.agents.get(agent_id=agent_id)

# Retrieve available interfaces
interfaces = {}
for interface in agent.retrieve_agentic_interfaces():
    name = getattr(interface, 'name', '').lower()
    interfaces[name] = interface

print(f"Found {len(interfaces)} available interfaces:")
for name in interfaces.keys():
    print(f"  - {name}")

Found 17 available interfaces:
  - social media
  - gmail
  - confluence
  - company monitor
  - similarweb
  - live sports data
  - hackernews
  - reddit
  - movie database
  - arxiv
  - xpanderai tools
  - zillow
  - amazon retail data
  - weather
  - ruben's personal google calendar
  - market insights
  - firecrawl


Each connector comes with its own set of operations or tools that we can freely use within our agent. These operations are designed for specific tasks, like retrieving an event ID, reading an event’s details, updating an event, and more.

Our AI agent can call one or more of these operations as part of its workflow to complete a given task. For example, let’s take a look at the various operations available through the Google Calendar connector.

In [5]:

for name, intf in interfaces.items():
    if "google calendar" in name:
        for operation in agent.retrieve_agentic_operations(agentic_interface=intf):
            name = getattr(operation, 'name', '')
            print(name)

Create New Secondary Calendar
Get Calendar Metadata by ID
Replace Calendar Metadata by ID
Delete Secondary Calendar by ID
Update Calendar Metadata by ID
Get Calendar ACL Rules by ID
Create Calendar ACL Rule by ID
Watch Calendar ACL Changes
Get ACL Rule by ID
Replace ACL Rule by ID
Delete ACL Rule by ID
Update ACL Rule by ID
Clear Primary Calendar by ID
Get Calendar Events by ID
Create Event in Calendar by ID
Import Event to Calendar by ID
Quick Add Event to Calendar by ID
Watch Calendar Events by ID
Get Event by ID
Replace Event by ID
Delete Event by ID
Update Event by ID
Get Recurring Event Instances
Move Event to Another Calendar
Stop Watching Resources
Get Color Definitions
Get Free Busy Info
Get User Calendar List Entries
Add Calendar List Entry
Watch Calendar List Changes
Get Calendar List Entry by ID
Replace Calendar List Entry by ID
Delete Calendar List Entry by ID
Update Calendar List Entry by ID
Get All User Settings
Watch User Settings Changes
Get User Setting by ID


The Google Calendar connector alone offers a wide range of operations, such as updating calendar metadata, creating events, importing events, and more. You can easily incorporate any of these into your agent as needed.

Aside from offering connectors to popular SaaS platforms, xpander also provides a collection of handy built-in actions. Let’s take a look at these built-in actions:

In [6]:
for name, intf in interfaces.items():
    if "xpanderai tools" in name:
        for operation in agent.retrieve_agentic_operations(agentic_interface=intf):
            name = getattr(operation, 'name', '')
            print(name)

Fetch Company Details by Query
Fetch Latest News
Fetch Stock Data by Symbol
Fetch Tavily AI Insights
Fetch Sports Events by Week
Check Recorder Status
Upload Text to File in S3
Upload File to S3 with Multipart Form
Send Email with Content
Convert Markdown to PDF
Perform OCR on File URL
Fetch Company Mentions on Social Media
Shorten URL with Expiration
Transcribe Audio Using URL
Create Meeting Recording Bot


As you can see, xpander offers a wide range of built-in actions, such as recording a meeting, sending content via email, uloading file to S3, etc into your workflow.

In this project, we’re going to add four different operations/actions to our AI agent:

- `CalendarEventManagementGetCalendarEventsById` (Google Calendar): Retrieves the event ID for the relevant meeting on Google Calendar.

- `XpanderMeetingManagementCreateRecordingBot` (xpander built-in action): Initializes and sets up the recording bot for the specified meeting ID.

- `XpanderMeetingManagementCheckRecorderStatus` (xpander built-in action): Checks the current status of the recording bot—whether it has joined the meeting or completed the recording. If completed, it returns the link to the recorded session.

- `XpanderMessagingServiceSendEmailWithContent` (xpander built-in action): Sends content to a specified email address based on the given prompt.

All of these operations are already listed in the `agent_config.json` configuration file included in this repo. Let’s go ahead and load them into our agent.



In [7]:
# Load configuration from file
config_path = "agent_config.json"  # You can change this to a custom path

try:
    with open(config_path, "r") as f:
        config = json.load(f)
    
    agent_name = config.get("agent_name")
    tools = config.get("tools", [])
    print(f"Using config file: {config_path}")
    print(f"Agent name: {agent_name}")
    print(f"Number of tools: {len(tools)}")
    
    # Display the tools configuration
    for i, tool in enumerate(tools, 1):
        print(f"\nTool {i}:")
        print(f"  Interface: {tool.get('interface')}")
        print(f"  Internal Name: {tool.get('internal_name')}")
except (FileNotFoundError, json.JSONDecodeError) as e:
    print(f"❌ Error loading config: {e}")

Using config file: agent_config.json
Agent name: AI Meeting Recorder Agent
Number of tools: 4

Tool 1:
  Interface: xpanderai tools
  Internal Name: XpanderMeetingManagementCreateRecordingBot

Tool 2:
  Interface: xpanderai tools
  Internal Name: XpanderMeetingManagementCheckRecorderStatus

Tool 3:
  Interface: xpanderai tools
  Internal Name: XpanderMessagingServiceSendEmailWithContent

Tool 4:
  Interface: google calendar
  Internal Name: CalendarEventManagementGetCalendarEventsById


Now, let’s take a look at the operations currently accessible to our agent:


In [8]:
# Get existing tools
existing_tools = {}
tools_list = agent.get_tools()
for tool in tools_list:
    if isinstance(tool, dict) and 'function' in tool and isinstance(tool['function'], dict):
        fn = tool['function']
        name = fn.get('name')
        if name:
            existing_tools[name] = True
print(f"Found {len(existing_tools)} existing tools:")
for name in existing_tools.keys():
    print(f"  - {name}")

Found 1 existing tools:
  - xpfinish-agent-execution-finished


As you can see, in its current state, our agent has access to only one tool: the `xpfinish-agent-execution-finished` operation from xpander. This operation is automatically initialized when the agent is created, and its main purpose is to signal that the agent has everything it needs to answer a query—meaning no further tool calls are necessary.

Now, let’s add the four operations we discussed earlier to our agent:

In [9]:
# Process tools from config
to_add = []
missing = []

for tool_config in tools:
    interface_name = tool_config.get("interface", "").lower()
    internal_name = tool_config.get("internal_name", "")
    
    if not internal_name:
        print(f"⚠️ Missing internal_name in tool config")
        continue
        
    if internal_name in existing_tools:
        print(f"✓ Tool already exists: {internal_name}")
        continue
    
    # Find interface (case insensitive partial match)
    interface = None
    for name, intf in interfaces.items():
        if interface_name in name:
            interface = intf
            print(f"Found suitable in your organization : {name}")
            break
            
    if not interface:
        print(f"⚠️ Interface not found: {interface_name}")
        missing.append(internal_name)
        continue
        
    try:
        ## Finds the operations for the interface in your specific organization
        operations = agent.retrieve_agentic_operations(agentic_interface=interface)
        
        ## Trying to match between the operations and the requested operation name from the agent config
        operation = None
        operation = next((op for op in operations if hasattr(op, 'id_to_use_on_graph') and op.id_to_use_on_graph == internal_name), None)
        if operation:
            print(f"+ Found tool by internal name: {internal_name}")
            print(f"+ Will add tool: {operation.name}")
            to_add.append(operation)
        else:
            print(f"❌ Operation not found with {internal_name}")
            missing.append(internal_name)
    except Exception as e:
        print(f"Error retrieving operations: {e}")
        missing.append(internal_name)

Found suitable in your organization : xpanderai tools
+ Found tool by internal name: XpanderMeetingManagementCreateRecordingBot
+ Will add tool: Create Meeting Recording Bot
Found suitable in your organization : xpanderai tools
+ Found tool by internal name: XpanderMeetingManagementCheckRecorderStatus
+ Will add tool: Check Recorder Status
Found suitable in your organization : xpanderai tools
+ Found tool by internal name: XpanderMessagingServiceSendEmailWithContent
+ Will add tool: Send Email with Content
Found suitable in your organization : ruben's personal google calendar
+ Found tool by internal name: CalendarEventManagementGetCalendarEventsById
+ Will add tool: Get Calendar Events by ID


In [10]:
# Add tools if needed
if to_add:
    try:
        print(f"adding {len(to_add)} tools to the agent")
        agent.attach_operations(operations=to_add)
        for op in to_add:
            agent.graph.add_node(GraphItem(
                agent=agent, item_id=op.id_to_use_on_graph, 
                name=op.name, type=AgentGraphItemType.TOOL, is_local_tool=False
            ))
        agent.sync()
        print(f"✅ Successfully added {len(to_add)} tools")
    except Exception as e:
        print(f"❌ Error adding tools: {e}")
else:
    print("No tools to add or skipping for demonstration")

adding 4 tools to the agent
✅ Successfully added 4 tools


Now let's see the graph visualization of our agent after adding these 4 operations and tools into our agent:

In [11]:
# Print summary
print(f"\nAgent setup complete!")
print(f"Go to: https://app.xpander.ai/agents/{agent.id}")


Agent setup complete!
Go to: https://app.xpander.ai/agents/75afc87c-5f62-40ab-8c5e-261fb3f4a654


If you visit the URL provided above, you should see a graph visualization similar to the one below:

![img_6](images/img_6.png)

As you can see, our agent now has access to four different operations from Google Calendar and xpander’s built-in actions. In the next notebook, we’ll run the agent to fetch our meeting schedule from Google Calendar, record the meeting, and send the recording to a specified email address.